In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import networkx as nx
import numpy.fft
import random
import bct

from source.utils.io_utils import readMRIFile, load_txt
from source.utils.matrix_utils import (
    createCorrelationMatrix,
    findThreshold,
    binarize,
    randomizeCorrelationMatrix,
    null_covariance,
    createNetwork,
    binarizeWithOutThreshold
)

from source.utils.plot_utils import plot_timeseries, plot_correlation_matrix
from source.utils.flexibility_utils import *

 #constant for our research purposes
#needs to be the path to where the data is
data_path = '/Users/nivaranavat/BrainConnectivity_ResearchProject/Data'

In [2]:
# #create our random time series
#utils.getRandomTimeSeries(path, min_freq = 0.01, max_freq = 0.1, samples = 200, samplerate =2, amt = 12)

In [3]:
# #phase scramble the drugs that we have
# #calculate the small worldness of each brain observed with the scrambled brain
# #need to figure out a threshold 

# drug_list = ("SAL", "COC", "MDPV")
# utils.phaseScrambled(drug_list, path, filesize = 200, iterations = 5, threshold = 0.15)


In [ ]:
# --- Calculate flexibility metrics for each dataset and plot results ---

# List of all drug/network conditions to analyze
drug_list = (
    "RANDOM", "SAL", "OrgSCRSAL", "SCRSAL", "COC", "OrgSCRCOC", "SCRCOC",
    "MDPV", "OrgSCRMDPV", "SCRMDPV"
)

# Plotting ranges for each drug/condition, for mean, coefficient of variation (cov), and standard deviation (sdv)
plot_ranges = {
    "RANDOM":     {"mean": [0, 0.11],  "cov": [0.15, 2],   "sdv": [0, 0.04]},
    "SAL":        {"mean": [0, 0.11],  "cov": [0.15, 2.6], "sdv": [0, 0.04]},
    "OrgSCRSAL":  {"mean": [0, 0.125], "cov": [0.15, 2],   "sdv": [0, 0.04]},
    "SCRSAL":     {"mean": [0, 0.11],  "cov": [0.15, 2],   "sdv": [0, 0.04]},
    "COC":        {"mean": [0, 0.11],  "cov": [0.15, 2.6], "sdv": [0, 0.04]},
    "OrgSCRCOC":  {"mean": [0, 0.125], "cov": [0.15, 2],   "sdv": [0, 0.04]},
    "SCRCOC":     {"mean": [0, 0.15],  "cov": [0.15, 2],   "sdv": [0, 0.04]},
    "MDPV":       {"mean": [0, 0.11],  "cov": [0.15, 2.6], "sdv": [0, 0.04]},
    "OrgSCRMDPV": {"mean": [0, 0.125], "cov": [0.15, 2],   "sdv": [0, 0.04]},
    "SCRMDPV":    {"mean": [0, 0.11],  "cov": [0.15, 2],   "sdv": [0, 0.04]},
}

# List of window sizes (in timepoints) to use for flexibility calculation
timepoints = [30, 60, 90, 120, 150]

# Output folder for saving results
folder_name = "results/timewindows[30,60,90,120,150]"

# Run the flexibility calculation for all drugs and time windows
# Returns: flexibility (dict of stats), fig (matplotlib Figure with boxplots)
flexibility, fig = calculate_flexibility(
    data_path, drug_list, timepoints, plot_ranges, folder_name
)

# Add a title to the figure summarizing the analysis parameters
fig.suptitle(
    "Flexibility's Standard Deviation, Coefficient of Variation, and Mean for each Drug "
    f"with time windows {timepoints}, splits of 50 and shifts of 1"
)

# Save the resulting figure to disk
fig.savefig("./results/FlexibilityPlots/flexibility__with_timepoints[30,60,90,120,150].png")


In [ ]:
#calculate flexibility of each every data set we have with the time shifts
drug_list = ("RANDOM","SAL", "OrgSCRSAL","SCRSAL","COC","OrgSCRCOC","SCRCOC", "MDPV","OrgSCRMDPV","SCRMDPV")
plot_ranges = {"RANDOM" : {"sdv" : [0,0.15], "cov" : [0.15, 2.5] , "mean": [0,0.35] } ,
              "SAL" : {"sdv" : [0,0.15], "cov" : [0.15, 5.25] , "mean": [0,0.35] }, 
               "OrgSCRSAL" : {"sdv" : [0,0.15], "cov" : [0.15, 2.5] , "mean": [0,0.4] },
              "SCRSAL" : {"sdv" : [0,0.15], "cov" : [0.15, 2.5] , "mean": [0,0.35] },
              "COC": {"sdv" : [0,0.15], "cov" : [0.15, 5.25] , "mean": [0,0.35] },
               "OrgSCRCOC": {"sdv" : [0,0.15], "cov" : [0.15, 2.5] , "mean": [0,0.4] },
              "SCRCOC": {"sdv" : [0,0.15], "cov" : [0.15, 2.5] , "mean": [0,0.35] },
              "MDPV": {"sdv" : [0,0.15], "cov" : [0.15, 5.25] , "mean": [0,0.35] },
               "OrgSCRMDPV": {"sdv" : [0,0.15], "cov" : [0.15, 2.5] , "mean": [0,0.4] },
              "SCRMDPV": {"sdv" : [0,0.15], "cov" : [0.15, 2.5] , "mean": [0,0.35] }}

timepoints = 90
splits = 12
shifts = [1,3,5,7,10]
folder_name = "results/timeshifts[1,3,5,7,10]"
flexibility,fig, small_worldness = calculate_flexilibity_with_timeshifts_and_save(timepoints, splits, shifts, drug_list, data_path, plot_ranges, folder_name)
fig.suptitle(f"Flexibility's Standard Deviation, Coefficient of Variation, and Mean for each Drug with time windows {timepoints} splits of {splits} and shifts of {shifts}")
fig.savefig("./results/FlexibilityPlots/flexibility_with_timewindow90_timeshifts[1,3,5,7,10].png")


In [ ]:
#calculate flexibility of each every data set we have with the time shifts
drug_list = ("RANDOM","SAL", "OrgSCRSAL","SCRSAL","COC","OrgSCRCOC","SCRCOC", "MDPV","OrgSCRMDPV","SCRMDPV")
plot_ranges = {"RANDOM" : {"sdv" : [0.03,0.1], "cov" : [0, 1] , "mean": [0.07,0.25] } ,
              "SAL" : {"sdv" : [0.03,0.1], "cov" : [0, 1] , "mean": [0.07,0.25] }, 
               "OrgSCRSAL" : {"sdv" : [0.03,0.1], "cov" : [0, 1] , "mean": [0.07,0.25] },
              "SCRSAL" : {"sdv" : [0.03,0.1], "cov" : [0, 1] , "mean": [0.07,0.25] },
              "COC": {"sdv" : [0.03,0.1], "cov" : [0, 1] , "mean": [0.07,0.25] },
               "OrgSCRCOC": {"sdv" : [0.03,0.1], "cov" : [0, 1] , "mean": [0.07,0.25] },
              "SCRCOC": {"sdv" : [0.03,0.1], "cov" : [0, 1] , "mean": [0.07,0.25] },
              "MDPV": {"sdv" : [0.03,0.1], "cov" : [0, 1] , "mean": [0.07,0.25] },
               "OrgSCRMDPV": {"sdv" : [0.03,0.1], "cov" : [0,1] , "mean": [0.07,0.25] },
              "SCRMDPV": {"sdv" : [0.03,0.1], "cov" : [0,1] , "mean": [0.07,0.25] }}

timepoints = 60
splits = 28
shifts = [5]
folder_name = "results/timeshift5_splits28"
#need to take the small worldness average here
flexibility,fig,small_worldness = calculate_flexilibity_with_timeshifts_and_save(timepoints, splits, shifts, drug_list, data_path,plot_ranges,folder_name, small_worldness_calc = True)
fig.suptitle(f"Flexibility's Standard Deviation, Coefficient of Variation, and Mean for each Drug with time windows {timepoints}, splits of {splits} and shifts of {shifts}")
fig.savefig("./results/FlexibilityPlots/flexibility_with_timewindow60_timeshifts5_28splits.png")

In [ ]:
print(f"Small Worldness with timeshifts {shifts} calculated with {timepoints} size time window")
    
import os
    
if not os.path.isdir("small_worldness"):
    os.makedirs("small_worldness")
for drug in drug_list:
    filename  = "small_worldness/" + drug + ".txt"
    with open(filename,"w") as f:
            for key, value in small_worldness[drug][0].items(): 
                f.write('%s:%s\n' % ( key,value[0][1]))

In [ ]:
from matplotlib import pyplot as plt
#draw out our last plot that is going to cover all the values calculated for the specified parameters
drug_sdv = {}
drug_cov = {}
drug_mean = {}

drug_names = {"SAL" : "Saline" , "COC" : "Cocaine", "MDPV" : "MDPV", "RANDOM": "Random"} #mapping of the abbreivated name to full name of drug

for drug, flex in flexibility.items():
    if drug in drug_names:
        name = drug_names[drug]
    if "SCR" in drug:
        name = "\u03BB'" + drug_names[drug.split("SCR")[1]]
    if "OrgSCR" in drug:
        name = "\u03BB''" + drug_names[drug.split("SCR")[1]]
    drug_sdv[name] = flex[0][5]
    drug_cov[name] = flex[1][5]
    drug_mean[name] = flex[2][5]

fig = plt.figure(figsize = (15,15))
box_plot(drug_sdv, (1,1,1), fig, "", "\u03C3", [0.02,0.1])
fig.savefig("./results/FlexibilityPlots/sdv_window60_shift5.png")
    

fig = plt.figure(figsize = (15,15))
box_plot(drug_cov, (1,1,1), fig, "", "CoV", [0, 1])
fig.savefig("./results/FlexibilityPlots/cov_window60_shift5.png")


fig = plt.figure(figsize = (15,15))
box_plot(drug_mean, (1,1,1), fig, "", "\u03BC", [0,0.25])
fig.savefig("./results/FlexibilityPlots/mean_window60_shift5.png")
    